## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse
# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] evaluate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.2 MB/s eta 0:00:00


Set kmer and model parameters

In [ ]:
kmer = 6
model_used = "armheb/DNA_bert_" + str(kmer)
bs = 64
epochs = 4
warmup = 100
maxstp = 1000
lr = 8e-5

run_name = "run" + "1-2"

Set output path

In [ ]:
# set output path

path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"
# stdpath = path_prefix + "DNABERT/output/" + "optimization" + "/"

Mounted at /content/drive


Run model on next 10 datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import sys
import os

# set up dataframe to capture testing results

test_column_names = ['test_loss', 'test_accuracy', 'test_precision', 'test_recall',
                     'test_f1', 'test_matthews_correlation', 'test_runtime']
testing_results_df = pd.DataFrame(columns=test_column_names)

# initialize parameters

for fname in [
#  "BroadDnd41CtcfUniPk-ran.csv",
#  "BroadDnd41Ezh239875UniPk-ran.csv",
#  "BroadGm12878CtcfUniPk-ran.csv",
#  "BroadGm12878Ezh239875UniPk-ran.csv",
#  "BroadH1hescChd1a301218aUniPk-ran.csv",
#  "BroadH1hescCtcfUniPk-ran.csv",
#  "BroadH1hescEzh239875UniPk-ran.csv",
#  "BroadH1hescJarid1aab26049UniPk-ran.csv",
#  "BroadH1hescRbbp5a300109aUniPk-ran.csv",
#  "BroadHelas3CtcfUniPk-ran.csv",
#  "BroadHelas3Ezh239875UniPk-ran.csv",
#  "BroadHelas3Pol2bUniPk-ran.csv",
#  "BroadHepg2CtcfUniPk-ran.csv",
#  "BroadHepg2Ezh239875UniPk-ran.csv",
#  "BroadHmecCtcfUniPk-ran.csv",
#  "BroadHmecEzh239875UniPk-ran.csv",
#  "BroadHsmmCtcfUniPk-ran.csv",
#  "BroadHsmmEzh239875UniPk-ran.csv",
#  "BroadHsmmtCtcfUniPk-ran.csv",
#  "BroadHsmmtEzh239875UniPk-ran.csv",
#  "BroadHuvecCtcfUniPk-ran.csv",
#  "BroadHuvecEzh239875UniPk-ran.csv",
#  "BroadHuvecPol2bUniPki-ran.csv",
#  "BroadK562Chd1a301218aUniPk-ran.csv",
#  "BroadK562CtcfUniPk-ran.csv",
#  "BroadK562Ezh239875UniPk-ran.csv",
#  "BroadK562Hdac1sc6298UniPk-ran.csv",
#  "BroadK562Hdac2a300705aUniPk-ran.csv",
#  "BroadK562Hdac6a301341aUniPk-ran.csv",
#  "BroadK562P300UniPk-ran.csv",
#  "BroadK562Phf8a301772aUniPk-ran.csv",
#  "BroadK562Plu1UniPk-ran.csv",
#  "BroadK562Pol2bUniPk-ran.csv",
#  "BroadK562Rbbp5a300109aUniPk-ran.csv",
#  "BroadK562Sap3039731UniPk-ran.csv",
#  "BroadNhaCtcfUniPk-ran.csv",
#  "BroadNhaEzh239875UniPk-ran.csv",
#  "BroadNhdfadCtcfUniPk-ran.csv",
#  "BroadNhdfadEzh239875UniPk-ran.csv",
#  "BroadNhekCtcfUniPk-ran.csv",
#  "BroadNhekEzh239875UniPk-ran.csv",
#  "BroadNhekPol2bUniPk-ran.csv",
#  "BroadNhlfCtcfUniPk-ran.csv",
#  "BroadNhlfEzh239875UniPk-ran.csv",
#  "BroadOsteoblCtcfUniPk-ran.csv",
#  "HaibA549Atf3V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Bcl3V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Creb1sc240V0416102Dex100nmUniPk-ran.csv",
#  "HaibA549Ctcfsc5916Pcr1xDex100nmUniPk-ran.csv",
#  "HaibA549Ctcfsc5916Pcr1xEtoh02UniPk-ran.csv",
#  "HaibA549Elf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Ets1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Fosl2V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Foxa1V0416102Dex100nmUniPk-ran.csv",
#  "HaibA549GabpV0422111Etoh02UniPk-ran.csv",
#  "HaibA549GrPcr1xDex500pmUniPk-ran.csv",
#  "HaibA549GrPcr1xDex50nmUniPk-ran.csv",
#  "HaibA549GrPcr1xDex5nmUniPk-ran.csv",
#  "HaibA549GrPcr2xDex100nmUniPk-ran.csv",
#  "HaibA549NrsfV0422111Etoh02UniPk-ran.csv",
#  "HaibA549P300V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Pol2Pcr2xDex100nmUniPk-ran.csv",
#  "HaibA549Pol2Pcr2xEtoh02UniPk-ran.csv",
#  "HaibA549Sin3ak20V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Six5V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Taf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Tcf12V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Usf1Pcr1xDex100nmUniPk-ran.csv",
#  "HaibA549Usf1Pcr1xEtoh02UniPk-ran.csv",
#  "HaibA549Usf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Yy1cV0422111Etoh02UniPk-ran.csv",
#  "HaibA549Zbtb33V0422111Etoh02UniPk-ran.csv",
#  "HaibEcc1CtcfcV0416102Dm002p1hUniPk-ran.csv",
#  "HaibEcc1EraaV0416102Bpa1hUniPk-ran.csv",
#  "HaibEcc1EralphaaV0416102Est10nm1hUniPk-ran.csv",
#  "HaibEcc1EralphaaV0416102Gen1hUniPk-ran.csv",
#  "HaibEcc1Foxa1sc6553V0416102Dm002p1hUniPk-ran.csv",
#  "HaibEcc1GrV0416102Dex100nmUniPk-ran.csv",
#  "HaibEcc1Pol2V0416102Dm002p1hUniPk-ran.csv"
]:

  dsname = path_prefix + "DNABERT/Datasets/tfbs/" + fname
  tfbs_dataset = pd.read_csv(dsname, sep=',')

# change standard error and output to saved output file

  stdpath = path_prefix + "DNABERT/output/" + fname.split(".")[0] + "/"

  try:
    os.makedirs(stdpath)
  except FileExistsError:
    pass

  stdpath_out = stdpath + "stdout.txt"
  stdpath_err = stdpath + "stderr.txt"

  sys.stdout = open(stdpath_out, 'w')
  sys.stderr = open(stdpath_err, 'w')

# print dataset shape and initial values

  print("tfbs dataset:", tfbs_dataset.shape, file=sys.stdout)
  print("tfbs dataset initial values:", tfbs_dataset.head(), file=sys.stdout)

# reformat input

  column_names = ["labels", "seq"]
  tfbs_dataset_res = pd.DataFrame(columns=column_names)

  j = 1
  for i in range(tfbs_dataset.count()['names']):
    name = tfbs_dataset['names'][i]
    pos_seq = tfbs_dataset['seq'][i]
    neg_seq = tfbs_dataset['neg_seq'][i]
    pos_label = 1
    neg_label = 0
    tfbs_dataset_res.loc[j] = [pos_label, pos_seq]
    tfbs_dataset_res.loc[j+1] = [neg_label, neg_seq]
    j+=2

  X = tfbs_dataset_res['seq']
  y = tfbs_dataset_res['labels']
  X_size = len(X)

# test range is 500 odd-numbered pairs of sequences
# training range is 500 even-numbered pairs plus remainder of the sequences over 1000

  test_range = list(range(3,1001,4)) + list(range(4,1001,4))
  test_range.sort()
  train_range = list(range(1,1001,4)) + list(range(2,1001,4))
  train_range.sort()
  train_range = train_range + list(range(1001,X_size))

  X_test = X.loc[test_range]
  X_train = X.loc[train_range]
  y_test = y.loc[test_range]
  y_train = y.loc[train_range]

# print size of training and test sets

  print("X_train shape:", X_train.shape, file=sys.stdout)
  print("y_train shape:", y_train.shape, file=sys.stdout)
  print("X_test shape:", X_test.shape, file=sys.stdout)
  print("y_test shape:", y_test.shape, file=sys.stdout)

# tokenization

  def kmers_stride1(s, k=kmer):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

# load pre-trained model

  model_cls = AutoModelForSequenceClassification.from_pretrained(model_used, num_labels=2)
  params = list(model_cls.named_parameters())
  tokenizer = AutoTokenizer.from_pretrained(model_used)

# reformat data to Hugging Face Dataset format from pandas

  ds_Xy_train = pd.concat([y_train, X_train], axis=1)
  ds_Xy_test = pd.concat([y_test, X_test], axis=1)

  Dataset_Xy_train = Dataset.from_pandas(ds_Xy_train)
  Dataset_Xy_test = Dataset.from_pandas(ds_Xy_test)
  Dataset_Xy_train, Dataset_Xy_test

  def tok_func(x): return tokenizer(" ".join(kmers_stride1(x["seq"])))

  Dataset_Xy_train_tok = Dataset_Xy_train.map(tok_func, batched=False)
  new_column = ["train"] * len(Dataset_Xy_train_tok)
  Dataset_Xy_train_tok = Dataset_Xy_train_tok.add_column("dset", new_column)

  Dataset_Xy_test_tok = Dataset_Xy_test.map(tok_func, batched=False)
  new_column = ["test"] * len(Dataset_Xy_test_tok)
  Dataset_Xy_test_tok = Dataset_Xy_test_tok.add_column("dset", new_column)

  dds = DatasetDict({
    'train': Dataset_Xy_train_tok,
    'test': Dataset_Xy_test_tok
  })

# switch to GPU

  if torch.cuda.device_count() > 0:
    model_cls.to('cuda')

# train model

  args = TrainingArguments('outputs', learning_rate=lr, warmup_steps=warmup, max_steps=maxstp, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

  def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy","precision","recall","f1","matthews_correlation"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  trainer.train();

# save model

  fpath = path_prefix + "DNABERT/Output_Models/" + fname.split(".")[0] + "/"
  print(fpath, file=sys.stdout)
  trainer.save_model(fpath)

# test model

  eval_preds = trainer.predict(dds['test'])

  print(eval_preds, file=sys.stdout)

  tfbs_ds = fname.split(".")[0] 
  testing_results_df.loc[tfbs_ds] = [eval_preds.metrics['test_loss'],
                                  eval_preds.metrics['test_accuracy'],
                                  eval_preds.metrics['test_precision'],
                                  eval_preds.metrics['test_recall'],
                                  eval_preds.metrics['test_f1'],
                                  eval_preds.metrics['test_matthews_correlation'],
                                  eval_preds.metrics['test_runtime']]
  
outpath = path_prefix + "DNABERT/output/" + run_name + ".csv"
testing_results_df.to_csv(outpath)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/77988 [00:00<?, ? examples/s]

Map:   0%|          | 0/25996 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265
4,0.117900,0.407823,0.889945,0.895473
